## Create a Masterlist of NYC subway stations and their GPS locations

1. Created a list of NYC station names (from file with Fares) -- DONE
2. Found a list of station names with GPS coordinates (be careful not to use station ENTRANCE file as the names are harder to match up) -- MAY BE ABLE TO FIND MORE COMPLETE LIST
3. Simple algorithm for matching between 1 and 2 above. 

About 1/3rd match right away.

About 1/3rd fail, but at least some could be rescued with LEVENSTEIN distances (I think). 
'KINGSTON - THROOP AVES' failed to find : 
['KINGSTON AVE-THROOP AVE',
'KINGSTON AVE/EASTERN PKWY']

About 1/3rd match too many times. See below for example:
['CANAL ST',
  'POINT (-74.00019299927328 40.71880300107709)',
  ['CANAL ST-LAFAYETTE ST EAST',
   'CANAL STREET',
   'CANAL STREET-BROADWAY',
   'CANAL STREET-LISPENARD ST',
   'CANAL STREET-VARICK STREET']

In [25]:
import pandas as pd
import numpy as np
import datetime

def nyc_stations():
    """returns list of nyc station names"""
    
    file = "Fare_Card_History_for_Metropolitan_Transportation_Authority__MTA___Beginning_2010.csv"
    df = pd.read_csv(file)
    df["Station"] = df.Station.str.strip()
    nyc_stations = list(set(df["Station"]))
    nyc_stations = sorted(nyc_stations)
    return nyc_stations

In [159]:
#the station names from the DataFrame with Fares
nyc_st = nyc_stations()
nyc_st


['103RD ST-CENTRAL PARK WEST',
 '103RD ST-ROOSEVELT AVE',
 '103RD STREET-BROADWAY',
 '103RD STREET-LEXINGTON AVE',
 '104TH STREET',
 '104TH STREET-LIBERTY AVENUE',
 '109TH STREET-LIBERTY AVENUE',
 '110TH STREET-8TH AVENUE',
 '110TH STREET-BROADWAY',
 '110TH STREET-LEXINGTON AVE',
 '111TH STREET',
 '111TH STREET-LENOX AVENUE',
 '111TH STREET-ROOSEVELT AVE',
 '116TH STREET-8TH AVENUE',
 '116TH STREET-COLUMBIA',
 '116TH STREET-LENOX AVENUE',
 '116TH STREET-LEXINGTON AVE',
 '121ST STREET',
 '125TH STREET',
 '125TH STREET & BROADWAY',
 '125TH STREET-LENOX AVENUE',
 '125TH STREET-LEXINGTON AVE',
 '135TH STREET-LENOX AVENUE',
 '135TH STREET-ST NICHOLAS AV',
 '137TH STREET-BROADWAY',
 '138TH ST-GRAND CONCOURSE',
 '138TH STREET-3RD AVE',
 '145TH STREET',
 '145TH STREET-BROADWAY',
 '145TH STREET-LENOX AVENUE',
 '148TH STREET-LENOX AVENUE',
 '149TH ST-GRAND CONCOURSE',
 '149TH STREET & 3RD AVENUE',
 '14TH STREET & 7TH AVENUE',
 '14TH STREET-6TH AVENUE',
 '14TH STREET-UNION SQUARE',
 '155TH STREET

In [155]:
#It is not clear how these stations match to our list...goint to try converting via geopy
#file = "DOITT_SUBWAY_ENTRANCE_01_13SEPT2010.csv"

#try a more generic subway station name to GPS coordinate file
file = "DOITT_SUBWAY_STATION_01_13SEPT2010.csv"
df = pd.read_csv(file)   
df2 = df.copy()
#df2
df2['NAME'] = df['NAME'].str.upper()
df2.head()

,URL,OBJECTID,NAME,the_geom,LINE,NOTES
0,http://web.mta.info/nyct/service/,1,ASTOR PL,POINT (-73.99106999861966 40.73005400028978),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
1,http://web.mta.info/nyct/service/,2,CANAL ST,POINT (-74.00019299927328 40.71880300107709),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
2,http://web.mta.info/nyct/service/,3,50TH ST,POINT (-73.98384899986625 40.76172799961419),1-2,"1-all times, 2-nights"
3,http://web.mta.info/nyct/service/,4,BERGEN ST,POINT (-73.97499915116808 40.68086213682956),2-3-4,"4-nights, 3-all other times, 2-all times"
4,http://web.mta.info/nyct/service/,5,PENNSYLVANIA AVE,POINT (-73.89488591154061 40.66471445143568),3-4,"4-nights, 3-all other times"


In [177]:
import re

no_match = []
one_match = []
many_match = []

for i, row in df2.iterrows() : 
    long_lat_re = re.compile(r'(-?\d*\.\d*) (-?\d*\.\d*)')
    matched = []
    matched = [s for i, s in enumerate(nyc_st) if row['NAME'] in s]
    if len(matched) == 1 :
        hits = long_lat_re.search(row.the_geom)
        one_match.append([row.NAME, hits.group(1), hits.group(2) , matched[0]])
    elif len(matched) == 0 :
        no_match.append(row.NAME)
    elif len(matched) > 1:
        many_match.append([row.NAME, row.the_geom, matched])
        
#one_match => 132
#no_match => 168
#many_match => 173

In [178]:
one_match[0:10]

[['ASTOR PL',
  '-73.99106999861966',
  '40.73005400028978',
  'ASTOR PLACE-4 AVENUE'],
 ['PENNSYLVANIA AVE',
  '-73.89488591154061',
  '40.66471445143568',
  'PENNSYLVANIA AVENUE'],
 ['65TH ST', '-73.8987883783301', '40.74971952935675', '65TH STREET-BROADWAY'],
 ['NORWOOD AVE',
  '-73.87962599910783',
  '40.68152000045683',
  'NORWOOD AVENUE-FULTON ST'],
 ['BEACH 90TH ST',
  '-73.81365140419632',
  '40.58809156457325',
  'HOLLAND-BEACH 90TH STREET'],
 ['FREEMAN ST',
  '-73.89175225349464',
  '40.829987446384116',
  'FREEMAN ST-SOUTHERN BLVD'],
 ['JUNCTION BLVD',
  '-73.86952700103515',
  '40.74914499948836',
  'JUNCTION BLVD-ROOSEVELT AVE'],
 ['BUHRE AVE',
  '-73.83256900003744',
  '40.846809998885504',
  'BUHRE AVE-WESTCHESTER AVE'],
 ['CASTLE HILL AVE',
  '-73.85122199961472',
  '40.83425499825462',
  'CASTLE HILL AVENUE'],
 ['ZEREGA AVE',
  '-73.8470359987544',
  '40.836488000608156',
  'ZEREGA AVE-WESTCHESTER AVE']]

In [158]:
no_match[0:10]

['CATHEDRAL PKWY (110TH ST)',
 'KINGSTON - THROOP AVES',
 'DELANCEY ST - ESSEX ST',
 '104TH-102ND STS',
 'BEACH 105TH ST',
 'INTERVALE AVE',
 '182ND-183RD STS',
 '174TH-175TH STS',
 'METS - WILLETS POINT',
 'FLUSHING - MAIN ST']

In [179]:
many_match[-20:]


[['CANAL ST',
  'POINT (-74.00105471306033 40.718814263587134)',
  ['CANAL ST-LAFAYETTE ST EAST',
   'CANAL STREET',
   'CANAL STREET-BROADWAY',
   'CANAL STREET-LISPENARD ST',
   'CANAL STREET-VARICK STREET']],
 ['23RD ST',
  'POINT (-73.99804100117201 40.74590599939995)',
  ['23RD STREET & 5TH AVENUE',
   '23RD STREET & 8TH AVENUE',
   '23RD STREET-6TH AVENUE',
   '23RD STREET-7TH AVENUE',
   '23RD STREET-ELY AVENUE',
   '23RD STREET-PARK AVE SOUTH']],
 ['14TH ST',
  'POINT (-74.00020100063497 40.737825999728116)',
  ['14TH STREET & 7TH AVENUE',
   '14TH STREET-6TH AVENUE',
   '14TH STREET-UNION SQUARE',
   '1ST AVENUE-14TH STREET',
   '3RD AVENUE-14TH STREET',
   '8TH AVENUE-14TH STREET',
   'PA-PATH 14TH STREET']],
 ['135TH ST',
  'POINT (-73.94753480879213 40.817905559212676)',
  ['135TH STREET-LENOX AVENUE', '135TH STREET-ST NICHOLAS AV']],
 ['14TH ST',
  'POINT (-73.99620899921355 40.73822799969515)',
  ['14TH STREET & 7TH AVENUE',
   '14TH STREET-6TH AVENUE',
   '14TH STREET-UN

In [153]:

len(many_match)

173

In [38]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="TR_test")
location = geolocator.geocode("167TH ST-GRAND CONCOURSE NYC")
location[-1]


(40.8345068, -73.9176949)

In [54]:
import pandas as pd
#Try getting station GPS from more complete list StationEntrances.csv
file = "StationEntrances.csv"
df = pd.read_csv(file)   
df3 = df.copy()
df4 = df3[['Station_Name', 'Station_Longitude', 'Station_Latitude']]
#print(df3.count)
df5 = df4.drop_duplicates(subset='Station_Name', keep='first')
#df6 = df5.reset_index()
#df6 = df5
#df6.index = df6.reset_index()
#len(df6['Station_Name'])
df5

,Station_Name,Station_Longitude,Station_Latitude
0,Ditmars Blvd,-73.912034,40.775036
4,Astoria Blvd-Hoyt Av,-73.917843,40.770258
8,30 Av-Grand Av,-73.921479,40.766779
12,Broadway,-73.925508,40.761820
15,36 Av-Washington Av,-73.929575,40.756804
18,39 Av-Beebe Av,-73.932755,40.752882
20,5th Av,-73.973347,40.764811
24,57th St,-73.980658,40.764664
28,49th St,-73.984139,40.759901
30,28th St,-73.988691,40.745494


## Failures

In [57]:
#this is rather slow lookup...
found = {}
not_found = []
for st in nyc_st[0:] :
    new = st + " NYC"
    location = geolocator.geocode(new)
    if location : 
        print("Located: " + new)
        found[st] = location[-1]
    else :
        print("Could not locate: " + new)
        not_found.append(new)


Located: 103RD ST-CENTRAL PARK WEST NYC
Located: 103RD ST-ROOSEVELT AVE NYC
Located: 103RD STREET-BROADWAY NYC
Could not locate: 103RD STREET-LEXINGTON AVE NYC
Located: 104TH STREET NYC
Located: 104TH STREET-LIBERTY AVENUE NYC
Could not locate: 109TH STREET-LIBERTY AVENUE NYC
Located: 110TH STREET-8TH AVENUE NYC
Could not locate: 110TH STREET-BROADWAY NYC
Could not locate: 110TH STREET-LEXINGTON AVE NYC
Located: 111TH STREET NYC
Could not locate: 111TH STREET-LENOX AVENUE NYC
Located: 111TH STREET-ROOSEVELT AVE NYC
Located: 116TH STREET-8TH AVENUE NYC
Could not locate: 116TH STREET-COLUMBIA NYC
Could not locate: 116TH STREET-LENOX AVENUE NYC
Could not locate: 116TH STREET-LEXINGTON AVE NYC
Located: 121ST STREET NYC
Located: 125TH STREET NYC
Located: 125TH STREET & BROADWAY NYC
Could not locate: 125TH STREET-LENOX AVENUE NYC
Could not locate: 125TH STREET-LEXINGTON AVE NYC
Could not locate: 135TH STREET-LENOX AVENUE NYC
Located: 135TH STREET-ST NICHOLAS AV NYC
Could not locate: 137TH ST

GeocoderServiceError: [Errno 65] No route to host

In [79]:
#Successful match => but actually just found 155th street (not the crossroad)
test1 = "155th STREET - 8th AVENUE NYC"
location = geolocator.geocode(test1)
location

Location(155th Street, Cedar Manor, Queens County, NYC, New York, 11433, USA, (40.695656, -73.798302, 0.0))

In [89]:
#Successful match => but actually just found 155th street (not the crossroad)
test1 = "155TH ST/8th Aveue NYC"
location = geolocator.geocode(test1)
location

In [74]:
#Failed match
test2 = '157TH STREET-BROADWAY NYC'
location = geolocator.geocode(test2)
location

In [75]:
#Failed => succesful match (removed the 'th' from 157th)
test2 = '157 STREET-BROADWAY NYC'
location = geolocator.geocode(test2)
location

Location(Broadway, Manhattanville, Manhattan, Manhattan Community Board 9, New York County, NYC, New York, 10031, USA, (40.8208309, -73.9547233, 0.0))